In [1]:
import numpy as np
import scipy as sp
import pandas as pd

from collections import defaultdict

import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [2]:
reviews = pd.read_csv('../data/all_reviews.csv',sep='\t',encoding='utf-8', index_col=0)
users = pd.read_csv('../data/all_users.csv',sep='\t',encoding='utf-8', index_col=0)

### Cleaning review data

In [3]:
reviews.animation = reviews.animation.str.strip().str[0:-3].convert_objects(convert_numeric=True)
reviews.character = reviews.character.str.strip().str[0:-3].convert_objects(convert_numeric=True)
reviews.overall = reviews.overall.str.strip().str[0:-3].convert_objects(convert_numeric=True)
reviews.sound = reviews.sound.str.strip().str[0:-3].convert_objects(convert_numeric=True)
reviews.story = reviews.story.str.strip().str[0:-3].convert_objects(convert_numeric=True)

### Cleaning user data

In [4]:
for i in users.index:
    temp = users.ix[i,'age_sex'].strip().split('/')
    age = '?'
    sex = '?'
    
    # check if user has provided age/sex
    if len(temp)==2:
        age = temp[0]
        sex = temp[1]  
    elif temp[0].isdigit():
        age = temp[0]
    else:
        sex = temp[0]
        
    users.ix[i,'age']=age
    users.ix[i,'sex']=sex
    
# remove those without age/sex
subusers = users.loc[(users.age!='? ')&(users.sex!='?')]
print subusers.shape
subusers.head()

,age_sex,join,place,age,sex
0Marcandre,20 / M,"Joined Mar 31, 2013",NaN,20,M
11bowman,21 / M,"Joined Jun 19, 2010","Amsterdam, Netherlands",21,M
11sam1234,21 / F,"Joined Nov 20, 2008",alto,21,F
29ActiveDownloads,20 / M,"Joined Apr 26, 2011",NaN,20,M
2minion,20 / F,"Joined Aug 2, 2013",NaN,20,F


### Combining user and review information

In [7]:
reviews.shape

(17194, 9)

In [18]:
# remove reviews whose user is not in user database
mask = [i in subusers.index for i in reviews.name]
subreviews = reviews.loc[mask]

# match age and sex info to reviews
age = []
sex = []

for i in subreviews.index:
    age.append(int(subusers.ix[subreviews.ix[i,'name'],'age']))
    sex.append(subusers.ix[subreviews.ix[i,'name'],'sex'])
    
subreviews.loc[:,'age']=age
subreviews.loc[:,'sex']=sex

subreviews.head()

,animation,anime,anime_name,character,name,overall,reviews,sound,story,age,sex
0,10,to-aru-majutsu-no-index-10th-anniversary-pv,To Aru Majutsu no Index 10th Anniversary PV,10,SogiitaGunhaGUTS,10.0,Well it's a short animation to celebrate Kamac...,NaN,NaN,15,M
1,5,dokkoida,Dokkoida?!,6,angelsreviews,5.0,NaN,5,8,26,F
2,9,inazuma-eleven,Inazuma Eleven,9,xxjacobxx3,9.0,first of all: I agree with ﻿﻿﻿﻿luminex's revie...,9,9,25,M
4,8,brigadoon-marin-to-melan,Brigadoon Marin to Melan,8,roriconfan,8.0,NaN,9,8,34,M
5,7,brigadoon-marin-to-melan,Brigadoon Marin to Melan,9,KiraRin,8.5,NaN,7,8,33,F


In [19]:
# replace missing information with random sex and average age
mean_age = subreviews.age.mean()

age = []
sex = []

final = reviews.copy()

for i in final.index:
    name = final.ix[i,'name']
    if name in subusers.index:
        age.append(int(subusers.ix[name,'age']))
        sex.append(subusers.ix[name,'sex'].strip())
    else:
        age.append(mean_age)
        ran = np.random.random_integers(0,high=1)
        if ran==1:
            sex.append('M')
        else:
            sex.append('F')
        
    
final.loc[:,'age']=age
final.loc[:,'sex']=sex

final.head()

,animation,anime,anime_name,character,name,overall,reviews,sound,story,age,sex
0,10,to-aru-majutsu-no-index-10th-anniversary-pv,To Aru Majutsu no Index 10th Anniversary PV,10,SogiitaGunhaGUTS,10,Well it's a short animation to celebrate Kamac...,NaN,NaN,15.00000,M
1,5,dokkoida,Dokkoida?!,6,angelsreviews,5,NaN,5,8,26.00000,F
2,9,inazuma-eleven,Inazuma Eleven,9,xxjacobxx3,9,first of all: I agree with ﻿﻿﻿﻿luminex's revie...,9,9,25.00000,M
3,8,inazuma-eleven,Inazuma Eleven,6,Lumineux,7,Story: 6,8,6,25.23511,F
4,8,brigadoon-marin-to-melan,Brigadoon Marin to Melan,8,roriconfan,8,NaN,9,8,34.00000,M


In [20]:
for i,val in enumerate(final.anime_name):
    if val == 'Beyblade: G Revolution':
        final.ix[i,'anime_name']='Beyblade G Revolution'
    if val == 'Beyblade: V-Force':
        final.ix[i,'anime_name']='Beyblade V-Force'
    if val == 'Beyblade: The Movie':
        final.ix[i,'anime_name']='Beyblade - The Movie'

In [21]:
# reordering columns
final = final[['anime', 'anime_name', 'name', 'age', 'sex', 'reviews', 'overall', 'animation', 'character', 'sound', 'story']]

### Adding category indicators

In [22]:
data=pd.read_csv("../data/anime_info.csv",sep='\t', index_col=0)
data.head()

,altname,atype,categories,date,date_pub,description,name,rank,rating,related,studio
(Making of) Evangelion: Another Impact,NaN,\n Web (1 ep x 6 min),"[Sci Fi, CG Animation, Japan Animator's Exhibi...",NaN,2015,"This episode is a making film of #12 ""evangeli...",(Making of) Evangelion: Another Impact,"\n Rank #5,996",1.743 of 5 from 153 votes,"[Neon Genesis Evangelion, Neon Genesis Evangel...",Studio Khara
+A-Channel,NaN,\nDVD Special (11 eps x 2 min),"[Comedy, Slice of Life, Based on a Manga]",NaN,2011,There are many untold stories in the lives of ...,+A-Channel,"\n Rank #2,398","3.334 of 5 from 1,192 votes","[A-Channel, A-Channel + smile]",Studio Gokumi
.hack//G.U. Returner,NaN,\nOVA (1 ep),"[MMORPG, RPG, Virtual Reality]",NaN,2007,"""There is something I wish to entrust to you; ...",.hack//G.U. Returner,"\n Rank #2,504","3.291 of 5 from 2,021 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN
.hack//G.U. Trilogy,NaN,\nMovie (1 ep x 93 min),"[CG Animation, MMORPG, RPG, Virtual Reality]",NaN,2008,The World is an ordinary online game – or at l...,.hack//G.U. Trilogy,"\n Rank #1,576","3.671 of 5 from 2,814 votes","[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN
.hack//G.U. Trilogy Parody,NaN,\nDVD Special (1 ep x 6 min),"[Comedy, MMORPG, RPG, Virtual Reality]",NaN,2008,NaN,.hack//G.U. Trilogy Parody,"\n Rank #3,993",2.691 of 5 from 714 votes,"[.hack//SIGN, .hack//LIMINALITY, .hack//Legend...",NaN


In [23]:
names = []
for i in data.index:
    names.append(i.encode('ascii', 'ignore'))
data.index=names

# checking how many unique categories
genres = data.categories.tolist()

temp = []
for i in genres:
    if type(i)==str:
        temp+=i.replace('[','').replace(']','').replace(' ','').split(',')
uniq_genres = list(set(temp))
len(uniq_genres)

233

In [24]:
# changing categories from string to list
temp = []
genres = data.categories.tolist()
for i in genres:
    if type(i)==str:
        temp.append(i.replace('[','').replace(']','').replace(' ','').split(','))
    else:
        temp.append([])
data.loc[:,'categories_list'] = temp

# finding categories
categories = defaultdict(list)
for i in final.anime_name:
    cat = data.loc[i.encode('ascii', 'ignore'),'categories_list']
    for j in cat:
        categories[j].append(1)
    for k in uniq_genres:
        if k not in cat:
            categories[k].append(0)
            
# add categories to final
for i in categories:
    final.loc[:,i] = categories[i]

In [25]:
per = final.shape[0]*0.005
mask = (final.ix[:,11:].sum(axis=0)>per)

for i in final.columns[11:]:
    check = mask[i]
    if not check:
        final.drop(i,axis=1,inplace=True)

In [26]:
final.to_csv('../data/final_review.csv')